**Redesigning An Engagement Algorithm to Make It Note Hateful**

In [1]:
# Example Content
example_content = [
    {
        'post': 'Example 1',
        'nuanced_language': True,
        'perspective_balance': 0.8,
        'perspective_range': 0.9,
        'conflict_potential': 0.1,
        'inflammatory_language': 0.1,
        'polarization_index': 0.2,
        'user_rating': 0.9
    },
    {
        'post': 'Example 2',
        'nuanced_language': False,
        'perspective_balance': 0.2,
        'perspective_range': 0.3,
        'conflict_potential': 0.8,
        'inflammatory_language': 0.7,
        'polarization_index': 0.9,
        'user_rating': 0.2
    },
    {
        'post': 'Example 3',
        'nuanced_language': True,
        'perspective_balance': 0.7,
        'perspective_range': 0.8,
        'conflict_potential': 0.2,
        'inflammatory_language': 0.2,
        'polarization_index': 0.3,
        'user_rating': 0.8
    }
]

*First we weigh the metrics to determine how important it is*

In [2]:
metrics = {
    'constructive_dialogue': 0.25, 
    'diversity_exposure': 0.1,        
    'community_health': 0.25,
    'user_reported_quality': 0.4
}

*These next few blocks are functions that measure the scores for each of the metrics I have above*

In [3]:
def measure_constructive_dialogue(content):
    dialogue_indicators = [
        content.get('nuanced_language', False),
        content.get('perspective_balance', 0) > 0.5,
        content.get('conflict_potential', 0) < 0.3
    ]
    return sum(dialogue_indicators) / len(dialogue_indicators)

In [4]:
def measure_diversity_exposure(content):
    return content.get('perspective_range', 0)

In [5]:
def measure_community_health(content):
    negative_impact_factors = [
        content.get('inflammatory_language', 0),
        content.get('polarization_index', 0)
    ]
    return 1 - max(negative_impact_factors)

In [6]:
def measure_user_reported_quality(content):
    return content.get('user_rating', 0.5)

*Now we calculate the overall score for a post!*

In [7]:
def calculate_content_score(content, metrics):
    scores = { 
        'constructive_dialogue': measure_constructive_dialogue(content),
        'diversity_exposure': measure_diversity_exposure(content),
        'community_health': measure_community_health(content),
        'user_reported_quality': measure_user_reported_quality(content)
    }
    
    total_score = sum(
        scores[metric] * weight 
        for metric, weight in metrics.items()
    )
    
    return max(0, min(1, total_score))

*This is where the 'ethical algorithm' comes into play by ranking all the content*

In [8]:
scored_content = [
    (content, calculate_content_score(content, metrics)) 
    for content in example_content
]

scored_content.sort(key=lambda x: x[1], reverse=True)

print("\n# Recommended Content Ranking:")
for content, score in scored_content:
    print(f"- {content['post']} (Ethical Engagement Score: {score:.2f})")


# Recommended Content Ranking:
- Example 1 (Ethical Engagement Score: 0.90)
- Example 3 (Ethical Engagement Score: 0.83)
- Example 2 (Ethical Engagement Score: 0.14)


*And here's a fun dashboard inspired by my work this summer*

In [9]:
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600&display=swap" rel="stylesheet">
        <style>
            .custom-slider .rc-slider-track {
                background-color: #22c55e;
            }
            .custom-slider .rc-slider-handle {
                border-color: #22c55e;
            }
            .custom-slider .rc-slider-handle:hover {
                border-color: #16a34a;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

COLORS = {
    'primary': '#22c55e',
    'secondary': '#64748b',
    'background': '#f0fdf4', 
    'card': '#ffffff',
    'border': '#dcfce7',
    'text': '#15803d',
    'muted': '#65a30d'
}

CARD_STYLE = {
    'backgroundColor': COLORS['card'],
    'borderRadius': '8px',
    'boxShadow': '0 1px 3px 0 rgba(0, 0, 0, 0.1)',
    'padding': '24px',
    'marginBottom': '24px',
}

example_content = [
    {"title": "Example 1", "nuanced_language": True, "perspective_balance": 0.8, 
     "perspective_range": 0.9, "conflict_potential": 0.1, "inflammatory_language": 0.1,
     "polarization_index": 0.2, "user_rating": 0.9},
    {"title": "Example 2", "nuanced_language": False, "perspective_balance": 0.2,
     "perspective_range": 0.3, "conflict_potential": 0.8, "inflammatory_language": 0.7,
     "polarization_index": 0.9, "user_rating": 0.2},
    {"title": "Example 3", "nuanced_language": True, "perspective_balance": 0.7,
     "perspective_range": 0.8, "conflict_potential": 0.2, "inflammatory_language": 0.2,
     "polarization_index": 0.3, "user_rating": 0.8}
]

default_metrics = {
    'constructive_dialogue': 0.3,
    'diversity_exposure': 0.2,
    'community_health': 0.3,
    'user_reported_quality': 0.2
}

def measure_constructive_dialogue(content):
    nuanced = 1.0 if content['nuanced_language'] else 0.0
    return (nuanced * 0.5 + content['perspective_balance'] * 0.5)

def measure_diversity_exposure(content):
    return content['perspective_range']

def measure_community_health(content):
    negative_factors = (content['conflict_potential'] + 
                       content['inflammatory_language'] + 
                       content['polarization_index']) / 3
    return 1 - negative_factors

def measure_user_reported_quality(content):
    return content['user_rating']

def normalize_weights(metrics):
    total = sum(metrics.values())
    if total == 0:
        return {k: 0 for k in metrics}
    return {k: v/total for k, v in metrics.items()}

def calculate_content_score(content, metrics):
    normalized_metrics = normalize_weights(metrics)
    
    scores = {
        'constructive_dialogue': measure_constructive_dialogue(content),
        'diversity_exposure': measure_diversity_exposure(content),
        'community_health': measure_community_health(content),
        'user_reported_quality': measure_user_reported_quality(content)
    }
    
    print(f"\nScoring {content['title']}:")
    print("Individual scores:", scores)
    print("Normalized weights:", normalized_metrics)
    
    weighted_sum = sum(scores[metric] * weight 
                      for metric, weight in normalized_metrics.items())
    
    print("Final weighted sum:", weighted_sum)
    
    return max(0, min(1, weighted_sum))

app.layout = html.Div([
    html.Div([
        html.H1("Content Ethical Engagement Dashboard",
                style={
                    'color': COLORS['text'],
                    'fontFamily': '"Inter", sans-serif',
                    'fontWeight': '600',
                    'fontSize': '2.25rem',
                    'textAlign': 'center',
                    'margin': '32px 0'
                })
    ]),
    
    html.Div([
        html.Div([
            html.H3("Metric Weights",
                    style={
                        'color': COLORS['text'],
                        'fontFamily': '"Inter", sans-serif',
                        'fontWeight': '600',
                        'marginBottom': '20px',
                        'textAlign': 'center'
                    }),
            html.Div([
                html.Div([
                    html.Label(
                        f"{metric.replace('_', ' ').title()}",
                        style={
                            'color': COLORS['text'],
                            'fontFamily': '"Inter", sans-serif',
                            'fontWeight': '500',
                            'marginBottom': '8px',
                            'display': 'block'
                        }
                    ),
                    dcc.Slider(
                        id=f'weight-{metric}',
                        min=0,
                        max=1,
                        value=weight,
                        step=0.1,
                        marks={i/10: {'label': str(i/10), 'style': {'color': COLORS['muted']}} 
                               for i in range(0, 11)},
                        className='custom-slider'
                    )
                ], style={'marginBottom': '24px'})
                for metric, weight in default_metrics.items()
            ])
        ], style=CARD_STYLE),
        
        html.Div([
            html.H3("Content Variables",
                    style={
                        'color': COLORS['text'],
                        'fontFamily': '"Inter", sans-serif',
                        'fontWeight': '600',
                        'marginBottom': '20px',
                        'textAlign': 'center'
                    }),
            dcc.Tabs(
                id='content-tabs',
                value='Example 1',
                children=[
                    dcc.Tab(
                        label=f"Example {i+1}",
                        value=f"Example {i+1}",
                        style={
                            'padding': '12px',
                            'fontFamily': '"Inter", sans-serif',
                        },
                        selected_style={
                            'padding': '12px',
                            'fontFamily': '"Inter", sans-serif',
                            'backgroundColor': COLORS['primary'],
                            'color': 'white'
                        },
                        children=[
                            html.Div([
                                html.Div([
                                    html.Label(
                                        "Nuanced Language",
                                        style={
                                            'color': COLORS['text'],
                                            'fontFamily': '"Inter", sans-serif',
                                            'fontWeight': '500',
                                            'marginBottom': '8px',
                                            'display': 'block'
                                        }
                                    ),
                                    dcc.RadioItems(
                                        id=f'nuanced-{i}',
                                        options=[
                                            {'label': 'True', 'value': True},
                                            {'label': 'False', 'value': False}
                                        ],
                                        value=content['nuanced_language'],
                                        style={'marginBottom': '24px'}
                                    )
                                ]),
                                *[
                                    html.Div([
                                        html.Label(
                                            f"{var.replace('_', ' ').title()}",
                                            style={
                                                'color': COLORS['text'],
                                                'fontFamily': '"Inter", sans-serif',
                                                'fontWeight': '500',
                                                'marginBottom': '8px',
                                                'display': 'block'
                                            }
                                        ),
                                        dcc.Slider(
                                            id=f'{var}-{i}',
                                            min=0,
                                            max=1,
                                            value=content[var],
                                            step=0.1,
                                            marks={i/10: {'label': str(i/10), 'style': {'color': COLORS['muted']}}
                                                   for i in range(0, 11)},
                                            className='custom-slider'
                                        )
                                    ], style={'marginBottom': '24px'})
                                    for var in ['perspective_balance', 'perspective_range',
                                              'conflict_potential', 'inflammatory_language',
                                              'polarization_index', 'user_rating']
                                ]
                            ], style={'padding': '20px'})
                        ]
                    ) for i, content in enumerate(example_content)
                ]
            )
        ], style=CARD_STYLE),
        
        html.Div([
            dcc.Graph(
                id='score-bar-chart',
                config={'displayModeBar': False}
            )
        ], style=CARD_STYLE)
    ], style={
        'maxWidth': '1200px',
        'margin': '0 auto',
        'padding': '0 24px'
    })
], style={
    'backgroundColor': COLORS['background'],
    'minHeight': '100vh',
    'paddingBottom': '32px'
})

@app.callback(
    Output('score-bar-chart', 'figure'),
    [Input(f'weight-{metric}', 'value') for metric in default_metrics.keys()] +
    [Input(f'nuanced-{i}', 'value') for i in range(len(example_content))] +
    [Input(f'{var}-{i}', 'value') 
     for i in range(len(example_content))
     for var in ['perspective_balance', 'perspective_range', 'conflict_potential',
                'inflammatory_language', 'polarization_index', 'user_rating']]
)
def update_chart(*values):
    metrics = dict(zip(default_metrics.keys(), values[:len(default_metrics)]))
    content_values = values[len(default_metrics):]
    updated_content = []
    
    for i in range(len(example_content)):
        base_idx = i * 7
        content = {
            'title': f'Example {i+1}',
            'nuanced_language': content_values[base_idx],
            'perspective_balance': content_values[base_idx + 1],
            'perspective_range': content_values[base_idx + 2],
            'conflict_potential': content_values[base_idx + 3],
            'inflammatory_language': content_values[base_idx + 4],
            'polarization_index': content_values[base_idx + 5],
            'user_rating': content_values[base_idx + 6]
        }
        updated_content.append(content)
    
    df = pd.DataFrame(updated_content)
    df['score'] = df.apply(lambda row: calculate_content_score(row, metrics), axis=1)
    
    fig = px.bar(df, 
                 x='title', 
                 y='score',
                 title='Content Ethical Engagement Scores')
    
    fig.update_layout(
        title={
            'text': 'Content Ethical Engagement Scores',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'family': 'Inter, sans-serif', 'size': 24, 'color': COLORS['text']}
        },
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        yaxis={
            'range': [0, 1],
            'tickformat': '.0%',
            'gridcolor': COLORS['border']
        },
        xaxis={
            'gridcolor': COLORS['border']
        },
        margin={'t': 60, 'b': 40, 'l': 40, 'r': 40},
        showlegend=False,
        height=400
    )
    
    fig.update_traces(
        marker_color=COLORS['primary'],
        hovertemplate='Score: %{y:.1%}<extra></extra>'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)